In [1]:
import torch

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def dcg(sims, labels, k=5):
    """
    sims and labels must have a same order
    sims: float, (n_g), predicted similarity scores(prediction)
    labels: int64, (n_g), {0(negative), 1(positive)}^n_g
    k: int, top-k
    """

    k = sims.size(0) if k < 1 or sims.size(0) < k else k
    _, idx = torch.topk(sims, k)
    labels = labels[idx].to(dtype=torch.float)

    # 1,2등에 점수 차이를 줄때, log(rank+1)
    disc_factor = torch.log2(torch.arange(k, dtype=torch.float)+2) # 1, 2등 차이 존재
    
    # 1,2등 점수 차이가 같고, log(rank) -- 1등과 2등이 같음
    # disc_factor = torch.log2(torch.arange(k, dtype=torch.float) + 1)  # 1, 2등을 같게
    # disc_factor[0] = 1
    
    dcg_val = (labels / disc_factor).sum()

    return dcg_val


def ndcg(sims, labels, k=5):
    """
    sims and labels must have a same order
    sims: float, (n_g), predicted similarity scores
    labels: float, (n_g), true relevance scores
    k: int, top-k
    """
    val = dcg(sims, labels, k=k)
    idcg = dcg(labels, labels, k=k)
    ndcg = 0 if idcg == 0 else val / idcg

    return ndcg

In [16]:
class nDCGMetric:
    def __init__(self, topK):
        self.topK = topK
        self.score = {f"nDCG@{k}": [] for k in self.topK}
            
    def update(self, pred, proxy):
        _, pred_idx = torch.topk(pred, max(self.topK))
        _, opt_idx = torch.topk(proxy, max(self.topK))
        
        for k in self.topK:
            pred_rel = proxy[pred_idx[:k]]
            opt_rel = proxy[opt_idx[:k]]
            
            # dcg = ((2**pred_rel - 1) / torch.log2(torch.arange(2, k+2))).sum()
            # idcg = ((2**opt_rel - 1) / torch.log2(torch.arange(2, k+2))).sum()
            dcg = (pred_rel / torch.log2(torch.arange(2, k+2))).sum()
            idcg = (opt_rel / torch.log2(torch.arange(2, k+2))).sum()
            
            
            self.score[f"nDCG@{k}"].append(dcg / idcg)
        
    def compute(self):
        return {
            k: torch.tensor(v).mean() for k, v in self.score.items()
        }
        
    def reset(self):
        self.score = {f"nDCG@{k}": [] for k in self.topK}
        
ndcg_metric = nDCGMetric([3])

In [17]:
pred = torch.tensor([0.5, 0.4, 0.3, 0.2, 0.1, 0.])
gt = torch.tensor([3., 2., 1., 3., 0., 0.])

In [18]:
ndcg_metric.update(pred, gt)
result = ndcg_metric.compute()
print(result)

{'nDCG@3': tensor(0.8081)}


In [65]:
result = ndcg(pred, gt)
print(result)

tensor(0.9574)
